## N = 15

### Set up Dataloader

In [1]:
import numpy as np

# load date
X = np.load("Datasets/kryptonite-15-X.npy")
X.shape

(30000, 15)

In [2]:
y = np.load("Datasets/kryptonite-15-y.npy")
y.shape

(30000,)

In [3]:
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader, Subset
from sklearn.model_selection import train_test_split, KFold


# Convert numpy arrays to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float)
y_tensor = torch.tensor(y, dtype=torch.float)
# create a TensorDataset
dataset = TensorDataset(X_tensor, y_tensor)

# define split sizes (90% train, 10% validation)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
# Split the dataset into train and validation
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])


### Set Up Model

In [4]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# build model for training
class MLP(nn.Module):
    def __init__(self, input_size=15, hidden_size=128, output_size=1):
        super(MLP, self).__init__()

        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, hidden_size*2)
        self.layer3 = nn.Linear(hidden_size*2, hidden_size*2)
        self.layer4 = nn.Linear(hidden_size*2, hidden_size)
        self.layer5 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.relu(self.layer3(x))
        x = F.relu(self.layer3(x))
        x = F.relu(self.layer4(x))
        x = torch.sigmoid(self.layer5(x))  # sigmoid activation for binary output
        return x


### Train

In [5]:
def train(model, train_loader, test_loader, loss_fn, optimizer, scheduler):
    torch.manual_seed(42)
    # set the number of epochs
    epochs = 60
    best_acc = 0

    for epoch in range(epochs):
        """ Training """
        model.train()

        # forward pass
        correct = 0
        total = 0
        train_loss = 0
        for X_batch, y_batch in train_loader:
            output = model(X_batch).squeeze()
            y_preds = torch.round(output)

            correct += torch.eq(y_preds, y_batch).sum().item()
            total += len(y_batch)

            loss = loss_fn(output, y_batch)
            train_loss += loss.item()
            # zero the optimizer
            optimizer.zero_grad()
            # backpropagattion
            loss.backward()
            # GD
            optimizer.step()

        scheduler.step()
        train_acc = (correct / total) * 100

        """ Testing """
        model.eval()
        correct = 0
        total = 0
        test_loss = 0
        with torch.inference_mode():
            for X_batch, y_batch in test_loader:
                output = model(X_batch).squeeze()
                y_preds = torch.round(output)

                correct += torch.eq(y_preds, y_batch).sum().item()
                total += len(y_batch)

                loss = loss_fn(output, y_batch)
                test_loss += loss.item()
            
            test_acc = (correct / total) * 100
            if test_acc > best_acc:
                best_acc = test_acc
                torch.save(model.state_dict(), "n-15best.pth")

        if epoch % 10 == 0:
            print(f"Epoch: {epoch} | Train Loss: {train_loss:.5f} | Acc: {train_acc:.2f}% | Learning Rate: {scheduler.get_last_lr()[0]:.7f} | Test loss: {test_loss:.5f} | Test Acc: {test_acc:.2f}%")
        
    return best_acc

In [6]:
# Set up 8-fold cross-validation on the training dataset
kf = KFold(n_splits=8, shuffle=True, random_state=42)
train_indices = list(range(len(train_dataset)))
batch = 128
accuracy = []
# Generate train-test splits
for fold, (train_idx, test_idx) in enumerate(kf.split(train_indices)):
    print("\n\nFold:", fold)
    """ create dataloaders for each fold """
    train_subset = Subset(train_dataset, train_idx)
    test_subset = Subset(train_dataset, test_idx)
    # Creating data loaders for each fold
    train_loader = DataLoader(train_subset, batch_size=batch, shuffle=True)
    test_loader = DataLoader(test_subset, batch_size=batch, shuffle=False)

    """ initialize the model, loss function, and optimizer """
    model = MLP(input_size=15, hidden_size=128, output_size=1)
    loss_fn = nn.BCELoss()  # binary Cross-Entropy Loss
    optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.1)

    """ train and test the model """
    acc = train(model, train_loader, test_loader, loss_fn, optimizer, scheduler)
    accuracy.append(acc)

print("\nAverage Accuracy is:", np.mean(accuracy))
    



Fold: 0
Epoch: 0 | Train Loss: 128.24710 | Acc: 49.83% | Learning Rate: 0.0001000 | Test loss: 18.71613 | Test Acc: 49.78%
Epoch: 10 | Train Loss: 127.65158 | Acc: 53.73% | Learning Rate: 0.0001000 | Test loss: 18.75695 | Test Acc: 49.93%
Epoch: 20 | Train Loss: 59.88523 | Acc: 87.62% | Learning Rate: 0.0001000 | Test loss: 9.32254 | Test Acc: 85.63%
Epoch: 30 | Train Loss: 39.09099 | Acc: 93.36% | Learning Rate: 0.0001000 | Test loss: 8.56876 | Test Acc: 87.53%
Epoch: 40 | Train Loss: 29.65409 | Acc: 95.63% | Learning Rate: 0.0000100 | Test loss: 5.91241 | Test Acc: 93.24%
Epoch: 50 | Train Loss: 28.19487 | Acc: 95.87% | Learning Rate: 0.0000100 | Test loss: 6.06411 | Test Acc: 92.89%


Fold: 1
Epoch: 0 | Train Loss: 128.23777 | Acc: 49.99% | Learning Rate: 0.0001000 | Test loss: 18.71546 | Test Acc: 49.93%
Epoch: 10 | Train Loss: 127.96831 | Acc: 52.28% | Learning Rate: 0.0001000 | Test loss: 18.73369 | Test Acc: 49.36%
Epoch: 20 | Train Loss: 101.90209 | Acc: 71.82% | Learning Rat

### Evaluation

In [8]:
model = MLP(input_size=15, hidden_size=128, output_size=1)
model.load_state_dict(torch.load("n-15best.pth"))
model.eval()

val_loader = DataLoader(val_dataset, batch_size=batch, shuffle=False)

correct = 0
total = 0
val_loss = 0
with torch.inference_mode():
    for X_batch, y_batch in tqdm(val_loader):
        output = model(X_batch).squeeze()
        y_preds = torch.round(output)

        correct += torch.eq(y_preds, y_batch).sum().item()
        total += len(y_batch)

        loss = loss_fn(output, y_batch)
        val_loss += loss.item()
    
    val_acc = (correct / total) * 100

print(f"Validation Loss: {val_loss:.5f} | Validation Acc: {val_acc:.2f}%")

100%|██████████| 24/24 [00:00<00:00, 1405.48it/s]

Validation Loss: 6.02529 | Validation Acc: 92.40%
